# PANDAPROSUMER CHP MODULE EXAMPLE

Example prepared by:
--------------------
    Izak Oberčkal Pluško*, Marko Keber*, Katja Klinar*, Tine Seljak*, Andrej Kitanovski*

    Faculty of Mechanical Engineering, University of Ljubljana

            *Contact: izak.oberckalplusko@fs.uni-lj.si 
            *Contact: marko.keber@fs.uni-lj.si
            *Contact: katja.klinar@fs.uni-lj.si
            *Contact: tine.seljak@fs.uni-lj.si
            *Contact: andrej.kitanovski@fs.uni-lj.si

DESCRIPTION:
--------------------
1 CHP unit connected directly to the consumer. CHP or combined heat and power plant is a system that simultaneously generates electricity and useful heat from the same source.

INSTRUCTIONS:
--------------------
1. Python 3.9 is required
2. Following libraries are required:
- astral==3.2
- converter==1.0.0
- coolprop==6.4.1
- matplotlib==3.7.1
- numpy==1.24.3
- openpyxl==3.1.2
- pandapipes==0.8.4
- pandapower==2.13.1
- pandas==2.0.2
- pyomo==6.6.1
- pyromat==2.2.4
- savReaderWriter==3.4.2
- scikit_learn==1.2.2
- scipy==1.10.1
- seaborn==0.12.2
- simbench==1.4.0
- tqdm==4.65.0
- writer==0.1.4


Before starting let's add a pandaprosumer parent directory:

In [137]:
import sys
import os

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)
print("Parent directory added to sys.path:", parent_directory)

INPUTS
--------------------

First let's import required libraries:


In [138]:
import pandas as pd
from pandapower.timeseries.data_sources.frame_data import DFData

We defined element properties - static input data for the chp element:

In [139]:
cycle = 't'
size = 0.35
name = 'example_chp'

We also define analysis time series:

In [140]:
start = '2020-01-01 00:00:00'                                              # time period
end = '2020-01-02 00:00:00'
resol = 900   

Now we import our demand data and transform it into appropriate DFData object:

In [141]:
demand_data = pd.read_excel('TestingData/input_chp.xlsx') # zdaj ni Markotovega excela
print(demand_data)
dur = pd.date_range(start, end, freq="15min")                              
demand_data.index = dur
data_source = DFData(demand_data)

CALCULATIONS
--------------------

First we define an empty prosumer container object which in practice can be anything:

In [142]:
from pandaprosumer.create import create_empty_prosumer_container
prosumer = create_empty_prosumer_container()

Then we define period of the analysis:

In [143]:
from pandaprosumer.create import define_period
period = define_period(prosumer, resol, start, end, 'utc', 'default')

Now we define a specific container which in this case is chp container where we pass into function empty prosumer container created above, static input data (cycle, size), additional parameter that is always set to True and the last static input data (name):

In [144]:
from pandaprosumer.create import define_ice_chp
chp_index = define_ice_chp(prosumer, size)
print(prosumer.ice_chp)

In [145]:
from pandaprosumer.create import define_heat_demand
heat_demand_index = define_heat_demand(prosumer, scaling=1.0)

In [146]:
from pandaprosumer.controller.data_model import ConstProfileControllerData
const_controller_data = ConstProfileControllerData(
    input_columns=['c', 'demand'],
    result_columns=["cycle_cp", "demand"],
    period_index = period
)
    
    

In [147]:
from pandaprosumer.controller.data_model.ice_chp import IceChpControllerData
ice_chp_controller_data = IceChpControllerData(element_name='ice_chp',                                                 # PM: copy of this here
        element_index=[chp_index],
        period_index=period
    )

In [148]:
from pandaprosumer.controller.data_model.heat_demand import HeatDemandControllerData
heat_demand_controller_data = HeatDemandControllerData(
        element_name='heat_demand',                                             
        element_index=[heat_demand_index],
        period_index=period
    )

In [149]:
from pandaprosumer.controller import ConstProfileController, HeatDemandController, IceChpController
ConstProfileController(prosumer,                                           
                           const_object=const_controller_data,                
                           df_data=data_source,                            
                           order=0,                                     
                           level=0)                                       
                                                                               
IceChpController(prosumer,
                     ice_chp_controller_data,
                     order=0,                                                   
                     level=1)                                                  

HeatDemandController(prosumer,                                              
                         heat_demand_controller_data,
                         order=0,
                         level=2)  # dont print this

![kuzek](TestingData/nemški-ovčar-1.webp)

In [150]:
from pandaprosumer.mapping import GenericMapping
GenericMapping(prosumer,                                                   
                   initiator=0,                                           
                   initiator_column="cycle_cp",                              
                   responder=1,                                                
                   responder_column="cycle")                                 

GenericMapping(prosumer,                                              
                   initiator=1,                                             
                   initiator_column="p_th_mw",
                   responder=2,                                           
                   responder_column="q_received_kw")

GenericMapping(prosumer,                                                  
                   initiator=0,                                               
                   initiator_column="demand",
                   responder=2,                                                
                   responder_column="q_demand_kw") # dont print this

In [151]:
from pandaprosumer.run_time_series import run_timeseries
run_timeseries(prosumer, period, True)

In [153]:
print(prosumer.heat_demand)
print(prosumer.time_series.data_source.loc[0].df.p_el_mw.plot())
import matplotlib.pyplot as plt
plt.show()